# Conservative Updates in a Count-Min Sketch

http://web.stanford.edu/class/cs168/p1.pdf

In [1]:
import hashlib

def get_hash(x, trial, table):
    md5 = hashlib.md5((str(x) + str(trial)).encode('utf-8')).hexdigest()
    substr = md5[2*table:2*table+2]
    return int(substr, 16)

assert get_hash(100, 8, 3) == 95

In [2]:
def get_true_occurrences(x):
    assert x >= 1
    assert x <= 9050
    if x <= 9000:
        return int((x - 1) / 1000) + 1
    return (x - 9000) ** 2

assert get_true_occurrences(1) == 1
assert get_true_occurrences(1000) == 1
assert get_true_occurrences(1001) == 2
assert get_true_occurrences(2000) == 2
assert get_true_occurrences(2001) == 3
assert get_true_occurrences(9000) == 9
assert get_true_occurrences(9001) == 1
assert get_true_occurrences(9002) == 4
assert get_true_occurrences(9050) == 2500

In [3]:
import random

RANGE = range(1, 9051)

def get_stream(order):
    if order == 'forward':
        res = []
        for i in RANGE:
            res += [i] * get_true_occurrences(i)
        return res
    if order == 'reverse':
        res = get_stream('forward')
        res.reverse()
        return res
    if order == 'random':
        res = get_stream('forward')
        random.shuffle(res)
        return res
    raise Exception(f'Unknown order: {order}')

total = 0
for i in RANGE:
    total += get_true_occurrences(i)
print(f'Total: {total} numbers')
true_heavy_hitters = []
for i in RANGE:
    if get_true_occurrences(i) * 100 >= total:
        true_heavy_hitters.append(i)
        
print(f'Total heavy hitters: {len(true_heavy_hitters)}')
print(true_heavy_hitters)

Total: 87925 numbers
Total heavy hitters: 21
[9030, 9031, 9032, 9033, 9034, 9035, 9036, 9037, 9038, 9039, 9040, 9041, 9042, 9043, 9044, 9045, 9046, 9047, 9048, 9049, 9050]


In [4]:
NUM_TABLES = 4

def count_min_sketch(stream, trial, update):
    assert update in ['standard', 'conservative']
    tables = [[0] * 256] * NUM_TABLES
    for x in stream:
        hs = [get_hash(x, trial, table_idx) for table_idx in range(NUM_TABLES)]
        min_count = min([tables[table_idx][hs[table_idx]] for table_idx in range(NUM_TABLES)])
        for table_idx in range(NUM_TABLES):
            if tables[table_idx][hs[table_idx]] == min_count or update == 'standard':
                tables[table_idx][hs[table_idx]] += 1
    return tables

def estimate_occurrences(x, tables, trial):
    return min([tables[h][get_hash(x, trial, h)] for h in range(NUM_TABLES)])

def estimate_heavy_hitters(tables, trial):
    res = []
    for i in RANGE:
        estimated_occurrences = estimate_occurrences(i, tables, trial)
        if estimated_occurrences * 100 >= total:
            res.append(i)
    return res

assert estimate_occurrences(9050, count_min_sketch(get_stream('forward'), 0, update='standard'), trial=0) == 3206
assert len(estimate_heavy_hitters(count_min_sketch(get_stream('forward'), 0, update='standard'), trial=0)) == 423

for update in ['standard', 'conservative']:
    for trial in range(10):
        for order in ['forward', 'reverse', 'random']:
            print(f'Trial #{trial + 1}/10, order={order}, update={update}')
            tables = count_min_sketch(get_stream(order), trial, update)
            print(f'Estimate for 9050: {estimate_occurrences(9050, tables, trial)}')
            estimated_heavy_hitters = estimate_heavy_hitters(tables, trial)
            print(f'Estimated heavy hitters: {len(estimated_heavy_hitters)}')
            #print(estimated_heavy_hitters)
            print()

Trial #1/10, order=forward, update=standard
Estimate for 9050: 3206
Estimated heavy hitters: 423

Trial #1/10, order=reverse, update=standard
Estimate for 9050: 3206
Estimated heavy hitters: 423

Trial #1/10, order=random, update=standard
Estimate for 9050: 3206
Estimated heavy hitters: 423

Trial #2/10, order=forward, update=standard
Estimate for 9050: 3126
Estimated heavy hitters: 392

Trial #2/10, order=reverse, update=standard
Estimate for 9050: 3126
Estimated heavy hitters: 392

Trial #2/10, order=random, update=standard
Estimate for 9050: 3126
Estimated heavy hitters: 392

Trial #3/10, order=forward, update=standard
Estimate for 9050: 3170
Estimated heavy hitters: 406

Trial #3/10, order=reverse, update=standard
Estimate for 9050: 3170
Estimated heavy hitters: 406

Trial #3/10, order=random, update=standard
Estimate for 9050: 3170
Estimated heavy hitters: 406

Trial #4/10, order=forward, update=standard
Estimate for 9050: 3155
Estimated heavy hitters: 382

Trial #4/10, order=reve